Loading Dataset




-Train and test data splitting


In [7]:
import pandas as pd

#creating train dataset using random rows from the combined dataset
df = pd.read_csv(r'G:\Diksha\7 sem\Final yearproject\NLP-text-doc-classification\Dataset\combined_final_dataset.csv')
df = df.sample(n=1367, random_state=1)
#save the csv to dataset folder
df.to_csv('combined_final_dataset_train.csv', index=False) #saving the training dataset


In [9]:
#creating test dataset using random rows from the combined dataset
df = pd.read_csv(r'G:\Diksha\7 sem\Final yearproject\NLP-text-doc-classification\Dataset\combined_final_dataset.csv')
df = df.sample(n=100, random_state=1)
#save in datset folder

df.to_csv('combined_final_dataset_test.csv', index=False) #saving the test csv


In [10]:
df_train = pd.read_csv('combined_final_dataset_train.csv')
df_test = pd.read_csv('combined_final_dataset_test.csv')